In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

input\tabular-playground-series-aug-2022\sample_submission.csv
input\tabular-playground-series-aug-2022\test.csv
input\tabular-playground-series-aug-2022\train.csv


In [2]:
test = pd.read_csv('input/tabular-playground-series-aug-2022/test.csv',
                    index_col='id')
display(test)

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
id,,,,,,,,,,,,,,,,,,,,,
26570,F,119.57,material_5,material_6,6,4,6,9,6,19.305,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
26571,F,113.51,material_5,material_6,6,4,11,8,0,17.883,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
26572,F,112.16,material_5,material_6,6,4,8,12,4,18.475,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
26573,F,112.72,material_5,material_6,6,4,8,11,10,16.518,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
26574,F,208.00,material_5,material_6,6,4,14,16,8,17.808,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,I,144.74,material_7,material_5,9,5,0,4,9,18.465,...,18.573,11.691,NaN,19.771,11.562,17.246,15.131,15.209,NaN,696.466
47341,I,74.53,material_7,material_5,9,5,4,8,7,18.900,...,17.917,10.980,16.027,15.694,13.564,15.494,15.296,13.812,16.501,613.249
47342,I,67.73,material_7,material_5,9,5,10,11,2,18.656,...,19.630,10.436,16.137,20.612,11.134,16.519,15.525,14.175,17.728,783.349


In [3]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import warnings
import csv
import joblib

In [4]:
#modify test data to fit the model
test_x = test.loc[:, ['loading', 'measurement_17', 'measurement_2', 'attribute_3','attribute_2','measurement_5','measurement_4']]
test_x['m_3_missing'] = test.measurement_3.isna()
test_x['m_5_missing'] = test.measurement_5.isna()
onehot = OneHotEncoder(categories=[['material_5', 'material_7']],
                    drop='first', sparse=False, handle_unknown='ignore')
onehot.fit(test[['attribute_0']])
test_x['onehot_att0'] = onehot.transform(test[['attribute_0']])
imputer = KNNImputer(n_neighbors=10)
imputer.fit(test_x)
test_x = imputer.transform(test_x)
test_x[:,0] = np.log1p(test_x[:,0])
test_x = np.column_stack((test_x, test_x[:, 3] * test_x[:, 4]))
test_x = np.delete(test_x, [3, 4], axis=1)


In [10]:
model = joblib.load('trained_model.joblib')

In [6]:
y_pred = model.predict_proba(test_x)[:,1]

In [7]:
#load test id 
test_data = []
with open(f'input/tabular-playground-series-aug-2022/sample_submission.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader)
    for row in csvreader:
        test_data.append(row)

In [8]:
#write predicted result into submission.csv
for i in range(len(y_pred)):
    test_data[i][1] = y_pred[i]

with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id","failure"])
    for t in test_data:
        writer.writerow(t)